# Lab | Text Generation from Shakespeare's Sonnet

This notebook explores the fascinating domain of text generation using a deep learning model trained on Shakespeare's sonnets. 

The objective is to create a neural network capable of generating text sequences that mimic the style and language of Shakespeare.

By utilizing a Recurrent Neural Network (RNN) with Long Short-Term Memory (LSTM) layers, this project aims to demonstrate how a model can learn and replicate the complex patterns of early modern English. 

The dataset used consists of Shakespeare's sonnets, which are preprocessed and tokenized to serve as input for the model.

Throughout this notebook, you will see the steps taken to prepare the data, build and train the model, and evaluate its performance in generating text. 

This lab provides a hands-on approach to understanding the intricacies of natural language processing (NLP) and the potential of machine learning in creative text generation.

Let's import necessary libraries

In [ ]:
!pip install tensorflow

In [ ]:
#conda install -c anaconda tensorflow

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np

Let's get the data!

In [ ]:
import requests
url = 'https://raw.githubusercontent.com/martin-gorner/tensorflow-rnn-shakespeare/master/shakespeare/sonnets.txt'
resp = requests.get(url)
with open('sonnets.txt', 'wb') as f:
    f.write(resp.content)

data = open('sonnets.txt').read()

corpus = data.lower().split("\n")

Step 1: Initialise a tokenizer and fit it on the corpus variable using .fit_on_texts

In [ ]:
# Your code here :
from tensorflow.keras.preprocessing.text import Tokenizer

# Step 1: Initialize a tokenizer
tokenizer = Tokenizer()

# Step 2: Fit the tokenizer on your corpus
tokenizer.fit_on_texts(corpus)

# To see the results:
word_index = tokenizer.word_index
print(f"Number of unique tokens: {len(word_index)}")
print("First 10 words and their indices:", list(word_index.items())[:10])

sequences = tokenizer.texts_to_sequences(corpus)
print("First 3 sequences:", sequences[:3])

Step 2: Calculate the Vocabulary Size

Let's figure out how many unique words are in your corpus. This will be the size of your vocabulary.

Calculate the length of tokenizer.word_index, add 1 to it and store it in a variable called total_words.

In [ ]:
# Your code here :
# Assuming you have already fit the tokenizer on the corpus as shown in previous examples

# Step 2: Calculate the Vocabulary Size
total_words = len(tokenizer.word_index) + 1

print(f"The total number of unique words in the corpus is: {total_words}")

Create an empty list called input_sequences.

For each sentence in your corpus, convert the text into a sequence of integers using the tokenizer.
Then, generate n-gram sequences from these tokens.

Store the result in the list input_sequences.

In [ ]:
# Your code here :
input_sequences = []

 

for line in corpus:

    # Convert each line into a sequence of integers

    token_list = tokenizer.texts_to_sequences([line])[0]

    for i in range(1, len(token_list)):

        # Generate n-grams for each sequence

        n_gram_sequence = token_list[:i+1]

        input_sequences.append(n_gram_sequence)

 

# Print first few sequences for verification

print(input_sequences[:5])  # Show the first 5 n-gram sequences

Calculate the length of the longest sequence in input_sequences. Assign the result to a variable called max_sequence_len.

Now pad the sequences using pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre').
Convert it to a numpy array and assign the result back to our variable called input_sequences.

Step-by-Step Guide:
Calculate max_sequence_len: Find the length of the longest sequence in input_sequences.
Pad Sequences: Use Keras's pad_sequences method to pad all sequences to the length of max_sequence_len.


In [ ]:
# Your code here :
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Assuming input_sequences is already defined and contains your sequences
# For example, let's say you have a list of sequences like this:
# input_sequences = [[1, 2, 3], [4, 5], [6, 7, 8, 9], ...]

# Step 1: Calculate the length of the longest sequence
max_sequence_len = max(len(seq) for seq in input_sequences)

# Step 2: Pad the sequences
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

# Convert to a numpy array (though `pad_sequences` already returns a numpy array)
input_sequences = np.array(input_sequences)

# Print to verify
print(f"The maximum sequence length is: {max_sequence_len}")
print("Padded sequences:\n", input_sequences)

Prepare Predictors and Labels

Split the sequences into two parts:

- Predictors: All elements from input_sequences except the last one.
- Labels: The last element of each sequence in input_sequences.

In [ ]:
x= input_sequences[:, :-1]
y= input_sequences [:, -1]

In [ ]:
print ("eje x:", x)
print ( "eje de y:", y)

In [ ]:
# Your code here :
# Assuming input_sequences is a numpy array containing your padded sequences

# Initialize lists to store predictors and labels
predictors = []
labels = []

# Iterate over each sequence
for sequence in input_sequences:
    predictors.append(sequence[:-1])  # All elements except the last one
    labels.append(sequence[-1])       # The last element

# Convert lists to numpy arrays for use in model training
predictors = np.array(predictors)
labels = np.array(labels)

# Print to verify
print("Predictors:\n", predictors)
print("Labels:\n", labels)

One-Hot Encode the Labels :

Convert the labels (which are integers) into one-hot encoded vectors. 

Ensure the length of these vectors matches the total number of unique words in your vocabulary.

Use ku.to_categorical() on labels with num_classes = total_words

Assign the result back to our variable labels.

In [ ]:
# Your code here :
from tensorflow.keras.utils import to_categorical  # Import the one-hot encoding function

# Assuming labels is a numpy array of your integer labels and total_words is defined as before

# One-hot encode the labels; num_classes should equal total number of unique words (vocabulary size)
labels = to_categorical(labels, num_classes=total_words)

# Print the shape to verify; it should be (number of sequences, total_words)
print("One-hot encoded labels shape:", labels.shape)

# Initialize the Model

Start by creating a Sequential model.

Add Layers to the Model:

Embedding Layer: The first layer is an embedding layer. It converts word indices into dense vectors of fixed size (100 in this case). Set the input length to the maximum sequence length minus one, which corresponds to the number of previous words the model will consider when predicting the next word.

Bidirectional LSTM Layer: Add a Bidirectional LSTM layer with 150 units. This layer allows the model to learn context from both directions (past and future) in the sequence. return_sequences=True

Dropout Layer: Add a dropout layer with a rate of 0.2 to prevent overfitting by randomly setting 20% of the input units to 0 during training.

LSTM Layer: Add a second LSTM layer with 100 units. This layer processes the sequence and passes its output to the next layer.

Dense Layer (Intermediate): Add a dense layer with half the total number of words as units, using ReLU activation. A regularization term (L2) is added to prevent overfitting.

Dense Layer (Output): The final dense layer has as many units as there are words in the vocabulary, with a softmax activation function to output a probability distribution over all words.

In [ ]:
model = Sequential([

    # Your code here :
    
])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.regularizers import l2

# Assuming total_words and max_sequence_len are already defined
embedding_dim = 100

# Initialize the model
model = Sequential([
    # Embedding Layer
    Embedding(input_dim=total_words, output_dim=embedding_dim, input_length=max_sequence_len - 1),
    
    # Bidirectional LSTM Layer
    Bidirectional(LSTM(150, return_sequences=True)),
    
    # Dropout Layer
    Dropout(0.2), # regularization to minimize overfitting by dropping out 20% of neurons randomly during training
        
    # LSTM Layer
    LSTM(100),
    
    # Dense Layer (Intermediate)
    Dense(total_words // 2, activation='relu', kernel_regularizer=l2(0.01)),  # L2 regularization for preventing overfitting
    
    # Dense Layer (Output)
    Dense(total_words, activation='softmax')
])

# Print the model summary to verify the layers
model.summary()

# Compile the Model:

Compile the model using categorical crossentropy as the loss function, the Adam optimizer for efficient training, and accuracy as the metric to evaluate during training.

In [ ]:
# Your code here :
# Compile the model
model.compile(loss='categorical_crossentropy',  # Suitable for multi-class classification
              optimizer='adam',                 # Efficient and commonly used optimizer
              metrics=['accuracy'])             # Track accuracy during training

# Verify the compilation
print("Model compiled successfully.")

# Print Model Summary:

Use model.summary() to print a summary of the model, which shows the layers, their output shapes, and the number of parameters.

In [ ]:
embedding_dim = 100  # Size of the dense vectors
max_sequence_len = 10  # Example length; replace with your actual max sequence length

model = Sequential([
    # Make sure the input_length is set
    Embedding(input_dim=total_words, output_dim=embedding_dim, input_length=max_sequence_len - 1),
    Bidirectional(LSTM(150, return_sequences=True)),
    Dropout(0.2),
    LSTM(100),
    Dense(total_words // 2, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(total_words, activation='softmax')
])

In [ ]:
# Your code here :
# Print the model summary
model.build (input_shape=(None, max_sequence_len -1)) # Correct input_shape specification when building the model manually
model.summary()

# Now train the model for 50 epochs and assign it to a variable called history.

Training the model with 50 epochs should get you around 40% accuracy.

You can train the model for as many epochs as you like depending on the time and computing constraints you are facing. Ideally train it for a larger amount of epochs than 50.

That way you will get better text generation at the end.

However, dont waste your time.

In [ ]:
# Assuming the model has been defined previously with layers added in the Sequential API.

# Step 1: Compile the model
model.compile(
    loss='categorical_crossentropy',  # Loss function suitable for one-hot encoded labels
    optimizer='adam',                 # Optimizer used for updating model parameters
    metrics=['accuracy']              # Metric for evaluating model performance
)

# Make sure the predictors and labels are properly prepared
# predictors should match the expected input shape (num_samples, max_sequence_len - 1)
# labels should be one-hot encoded, matching the total number of classes (vocabulary size)

# Step 2: Train the model
history = model.fit(
    predictors, 
    labels,
    epochs=50,
    batch_size=32,
    validation_split=0.2,  # Optionally use a portion of data for validation
    verbose=1
)

# Optionally, you can print or plot the history to check training progress
import matplotlib.pyplot as plt

# Plotting training accuracy
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history.get('val_accuracy', []), label='val_accuracy')  # Only if validation data is used
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

# Use plt from matplotlib to plot the training accuracy over epochs and the loss over epochs

First you will have to get the accuracy and loss data over epochs, you can do this by using methods on your model.

In [ ]:
# Your code here :
import matplotlib.pyplot as plt

# Assuming 'history' is the object returned by model.fit

# Extract the accuracy and loss data
accuracy = history.history['accuracy']
loss = history.history['loss']

# If you used a validation split, you can plot validation metrics as well
val_accuracy = history.history.get('val_accuracy', [])
val_loss = history.history.get('val_loss', [])

# Plot training accuracy and validation accuracy over epochs
plt.figure(figsize=(12, 6))  # Set figure size

# Plot training accuracy
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
plt.plot(accuracy, label='Training Accuracy')
# Plot validation accuracy if available
if val_accuracy:
    plt.plot(val_accuracy, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

# Plot training loss and validation loss over epochs
plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
plt.plot(loss, label='Training Loss')
# Plot validation loss if available
if val_loss:
    plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')

plt.tight_layout()  # Adjust subplots to fit the figure area.
plt.show()

# Generate text with the model based on a seed text

Now you will create two variables :

- seed_text = 'Write the text you want the model to use as a starting point to generate the next words'
- next_words = number_of_words_you_want_the_model_to_generate

Please change number_of_words_you_want_the_model_to_generate by an actual integer.

In [ ]:
# Your code here :
seed_text = "Tell me something about Shakspear's sonnets."
next_words = 20

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
seed_text

Now create a loop that runs based on the next_words variable and generates new text based on your seed_text input string. Print the full text with the generated text at the end.

This time you dont get detailed instructions.

Have fun!

In [ ]:
# Your code here :

Experiment with at least 3 different seed_text strings and see what happens!

In [ ]:
# Your code here :